# Engineering Notebook: Design Exercise 1

### Ideas for the Wire Protocol

- Use TCP/IP as template
    - Do we need IP addresses in the header?
    - How can we verify packets?
- VBS: Should we use some kind of compression for the message contents?
    - VBS: Maybe not, let's keep it simple
- Header (8 bytes):
    - Version (1 byte)
    - Request code (think: HTTP status code but also use this to specify the operation) (1 byte)
    - Flags/status (2 bytes) -> for the future, also padding
    - Packet checksum (2 bytes)
    - Payload length (2 bytes)

- What type of data are we sending across?
    - Operations + arguments (e.g. page/pagination, optional arguments)
    - Strings (e.g. name, password hash, error messages)
    - Status code (from the server to detect errors)
    - Arrays of strings -> could be sent as string of strings separated by commas like a CSV
        - VBS: we need to allow for commas in a string, should we just use ascii and then have a special non-ascii character for either both end of string and end of item or a separate one for end of item and end of string, kind of like '\0' in C?
        - Let's not use this. Let's use another, more flexible encoding instead
    - Assuming that we never need to send over dictionaries but at most lists, we can use an encoding for "fields" like this:
        - Field ID (1 byte), Field Length (2 bytes), Field Value (bytes)
        - This would mean that the client and server need to agree on which field ID corresponds to which field (e.g. username -> ID 1 etc.), and -- importantly -- what the *type* of each field is (e.g. float, string, unsigned int/int, list of float/string etc.); also, there are only 2^8 = 255 possible fields we can use which should be enough for this application but may cause issues in the future if there is a lot of data that we need to transmit
            - One possible workaround for this is to have an "OTHER" value kind of like in UTF-8 where if the first bit is set, you actually read 2 bytes for the field ID and use that as your index; this would be much more flexible but also more work to implement
        - Do we want to allow lists of different types? Probably not necessary
        - Side note: we can store the types efficiently using some byte flag for "list" and then different byte values for the type, and then check if "list & type" is 1 to detect that it's a list
        - We should agree that we're using little-endian for all communication
            - Is there any way to define flexible length types, e.g. specify "int" but then allow 8-byte integers?
    - This doesn't let us send lists of lists or lists of dictionaries or lists of a custom struct. Could be a problem when we're sending all unread messages, since this would probably be a list of lists or a list of dicts or a list of "Message"s; or rather, it would introduce more overhead because we first need to send back the IDs of the messages and then request each of them individually. Might be something we're willing to compromise on
        - We could do a similar thing as with the lists and define an "object" flag which is then interpreted as a dictionary with custom types; for this, we also need to allow lists of objects or more generally, lists of different types
        - This is fine in Python but could cause problems in other languages like C


- How do you specify which messages you want to delete?
    - Could go by message ID individually
    - Could go by sender/receiver name

### General Notes
- Each request should carry a token to indicate that the user is logged in and to map the request to the correct account. Do we refresh this token? How often? Does the server just send back the new token with each request? In that case, what do we do if a packet is lost; this would automatically log the user out

### User Interface Design

- How do we display the unread messages?
    - VBS: Maybe users want to be able to open the app without reading *any* messages, so we should only load messages on request
    - We can either group them by conversation or just give a stream of messages ordered by timestamp
    - Think: "Refresh mail" button in any standard Mail program
- To send a message, should users be able to click on a conversation and send a message there or do we just have two input fields, one for the user name and one for the message, and then you just have to input the user name every time

### Data Storage
- How do we want to store the data? Should we just keep stuff in memory, should we use an actual database (e.g. sqlite) or just keep stuff in one or multiple JSON files?

VBS says:

1. first check if account exists, if yes then check hash of password matches, (send string of hash and username over network); if no then client shows an error of account does not exist then allow to put in password (which client then hashes). if successful then always use username password hash as our token (extension is valid for some period of time)

2. basically the same as above; check if account exists and check if password match. also send back the number of unread messages

3. send search pattern, what page we are on, how big are the pages; 

4. specify the person through user ID or username, we need to keep track of the sockets belongs to which user, once destroyed also keep track of who is destroyed; we will store everything when it is not logged in, otherwise send immediately; get some form of confirmation all good back

5. send back bunch of mesages at once using the object thing; what if the message is super long? (think about this). be able to send a request to the server of how many messages we want, if there are none just send back something; 

6. delete a specific message - needs an ID then client interprets and checks if you can delete this message; also the similar for a list of messages; for a whole conversation - separate action for deleting conversation with a username/userID; delete account - can only delete your account, what happens to our messages? (policy: every message you send and sent to you are gone)

7. front end



To Do:

1. make a small library that takes data and turn into wireprotocl (bytes); and translate it back; probably use UTS-8 for strings interpretation

2. user interface, graphic interface for the client
- how do we display messages, unread messages gets added to the top, all previous read messages at the bottom(let us not do conversations)

- when you log in, you see all previous messages, and shows how many unread messages, and you can ask for some number of unread messages; do this in pages, 

3. server side + database storage


## Code Ideas

Some ideas for what the field ID to field mapping could look like so that the data becomes a bit easier to use for the programmer and we don't need to know all of the field IDs by heart:

In [ ]:
key_to_field_id = {
    "username": 0,
    "password_digest": 1,
}

key_to_field_type = {
    "username": str,
    "password_digest": str,
    "asdkjh": int,
}

def get(data, field: str, default = None):
    if field in key_to_field_id:
        return data[key_to_field_id[id]]
    
    return default

In [ ]:
protocol:
    field ID, field type, field size, field content
    1 byte, 

In [ ]:
sending: list of messages

In [ ]:
0x20 -> list
0x21 -> list of int
0x22 -> list of str

In [ ]:
class Message:
    id: int
    content: str
    from_id: int
    to_id: int
    timestamp: datetime

In [ ]:
{
    "version": 1,
    "operation": "list",
    "arguments": {
        "search": "hello"
    }
}

In [ ]:
what the request would look like with JSON:

request

{
    "version": 1,
    "operation": "messages",
    "arguments": {
        "number": 3
    }
}

response 

{
    "version": 1,
    "status": "success",
    "items": [
        {
            "message_id": 1,
            "sender": "user_1",
            "message": "hey",
            "timestamp": "2025-01-01"
        },
        {
            "message_id": 2,
            "sender": "user_1",
            "message": "what's up?",
            "timestamp": "2025-01-01"
        },
        {
            "message_id": 3,
            "sender": "user_2",
            "message": "yooo",
            "timestamp": "2025-01-01"
        }
    ],
    "remaining": 2
}


In [ ]:
what the same request would look like with our protocol:

request: ..., operation = messages, quantity = 3
response: ..., items = [1, 2, 3], remaining = 2
request: ..., operation = message_detail, id = 1
response: ..., message = ["user_1", "hey", "2025-01-01"]
request: ..., operation = message_detail, id = 2
response: ..., message = ["user_1", "what's up?", "2025-01-01"]
request: ..., operation = message_detail, id = 3
response: ..., message = ["user_2", "yooo", "2025-01-01"]

In [ ]:
{
    "messages": [
        {
            "id": 1,
            "content": "My message",
            "from_id": 2,
            "to_id": 3,
            "timestamp": "2025-01-01"
        }
    ],
    "users": [
        {
            "id": 
        }
    ]
}

In [ ]:
message = [
    "user_name",
    "message",
    "timestamp"
]

In [ ]:
request: give me the messages
response: here are the message IDs (all integers)
for each message ID:
    request: give me the message (message ID)
    response: here is the message (list of str)

In [ ]:
send("message", {"to": "test_user", "message": "hello"})

In [ ]:
"hello"

message -> 3
field_type -> str
field_length -> 5